In [7]:
import ast
import import_ipynb
import Extract_Python_Pairs as EPP
#https://docs.python.org/3/library/tokenize.html
import tokenize
import io
import numpy as np
from tqdm import tqdm #inline progress bar (quality of life)

In [3]:
#get the code and comment snippets from the Extract Python Pairs file
code_snippets, comments = EPP.get_python_pairs(EPP.get_all_data())

  0%|          | 15320/16115540 [00:00<01:45, 153191.02it/s]

Completed in 227 seconds


100%|██████████| 16115540/16115540 [00:36<00:00, 444800.20it/s] 


Total pairs: 2508330


In [ ]:
'''
ENDMARKER = 0
NAME = 1
NUMBER = 2
STRING = 3
NEWLINE = 4
INDENT = 5
DEDENT = 6
LPAR = 7
RPAR = 8
LSQB = 9
RSQB = 10
COLON = 11
COMMA = 12
SEMI = 13
PLUS = 14
MINUS = 15
STAR = 16
SLASH = 17
VBAR = 18
AMPER = 19
LESS = 20
GREATER = 21
EQUAL = 22
DOT = 23
PERCENT = 24
LBRACE = 25
RBRACE = 26
EQEQUAL = 27
NOTEQUAL = 28
LESSEQUAL = 29
GREATEREQUAL = 30
TILDE = 31
CIRCUMFLEX = 32
LEFTSHIFT = 33
RIGHTSHIFT = 34
DOUBLESTAR = 35
PLUSEQUAL = 36
MINEQUAL = 37
STAREQUAL = 38
SLASHEQUAL = 39
PERCENTEQUAL = 40
AMPEREQUAL = 41
VBAREQUAL = 42
CIRCUMFLEXEQUAL = 43
LEFTSHIFTEQUAL = 44
RIGHTSHIFTEQUAL = 45
DOUBLESTAREQUAL = 46
DOUBLESLASH = 47
DOUBLESLASHEQUAL = 48
AT = 49
ATEQUAL = 50
RARROW = 51
ELLIPSIS = 52
COLONEQUAL = 53
OP = 54
AWAIT = 55
ASYNC = 56
TYPE_IGNORE = 57
TYPE_COMMENT = 58
# These aren't used by the C tokenizer but are needed for tokenize.py
ERRORTOKEN = 59
COMMENT = 60
NL = 61
ENCODING = 62
N_TOKENS = 63
# Special definitions for cooperation with parser
NT_OFFSET = 256
'''

In [4]:
#https://docs.python.org/3/library/tokenize.html
def tokenize_python(code_snippet):
    tokens = tokenize.tokenize(io.BytesIO(code_snippet.encode('utf-8')).readline)
    parsed = []
    for token in tokens:
        if token.type not in [0,57,58,59,60,61,62,63,256]:
            #print('----')
            #print(token)
            parsed.append(token.string)
    return parsed

#EXAMPLE
print('Original:')
print(code_snippets[0])
print(len(code_snippets[0]))
print()
print('Tokenized:')
print(tokenize_python(code_snippets[0]))
print('number of tokens:',len(tokenize_python(code_snippets[0])))

Original:
  def __call__(mcs, *args, **kwargs):
    
    if kwargs:
      return type.__call__(mcs, *args, **kwargs)
    else:
      instances = mcs._instances
      key = (mcs,) + tuple(args)
      try:
        return instances[key]
      except KeyError:
        # No cache entry for key exists, create a new one.
        return instances.setdefault(key, type.__call__(mcs, *args))
      except TypeError:
        # An object in args cannot be hashed, always return
        # a new instance.
        return type.__call__(mcs, *args)



527

Tokenized:
['  ', 'def', '__call__', '(', 'mcs', ',', '*', 'args', ',', '**', 'kwargs', ')', ':', '\n', '    ', 'if', 'kwargs', ':', '\n', '      ', 'return', 'type', '.', '__call__', '(', 'mcs', ',', '*', 'args', ',', '**', 'kwargs', ')', '\n', '', 'else', ':', '\n', '      ', 'instances', '=', 'mcs', '.', '_instances', '\n', 'key', '=', '(', 'mcs', ',', ')', '+', 'tuple', '(', 'args', ')', '\n', 'try', ':', '\n', '        ', 'return', 'instances', '['

In [5]:
#small_code = code_snippets[:10000]
#small_comment =  comments[:10000]
small_code = code_snippets
small_comment =  comments

In [11]:
import nltk
#nltk.download('punkt')

In [12]:
Source_Sequences = [] #eng_sent = []
Target_Sequences = [] #fra_sent = []
Source_Tokens = set() #eng_chars = set()
Target_Tokens = set() #fra_chars = set()


errors = 0
for i in tqdm(range(len(small_code))):
    try:
        Eng = ''.join(ch for ch in small_comment[i] if ch.isalnum() or ch ==' ').lower()
        Eng_toks = nltk.word_tokenize(Eng)
        Target_Sequences.append(Eng_toks)

        python_toks = tokenize_python(small_code[i])
        Source_Sequences.append(python_toks)


        for nodeType in python_toks:
            if (nodeType not in Source_Tokens):
                Source_Tokens.add(nodeType)


        for word in Eng_toks:
            if (word not in Target_Tokens):
                Target_Tokens.add(word)
    except Exception as e:
        #print(e)
        errors+=1

print('number of errors:',errors,'or approixmatiely ',(errors/len(small_code))*100,'% of samples')
    

100%|██████████| 2508330/2508330 [16:36<00:00, 2517.97it/s]

number of errors: 74483 or approixmatiely  2.9694258729911933 % of samples


In [ ]:
Source_Tokens = sorted(list(Source_Tokens))
Target_Tokens = sorted(list(Target_Tokens))
nb_samples= len(Source_Sequences) #for all

# dictionary to index each english character - key is index and value is english character
source_index_to_token_dict = {}

# dictionary to get english character given its index - key is english character and value is index
source_token_to_index_dict = {}

for k, v in enumerate(Source_Tokens):
    source_index_to_token_dict[k] = v
    source_token_to_index_dict[v] = k

# dictionary to index each french character - key is index and value is french character
target_index_to_token_dict = {}

# dictionary to get french character given its index - key is french character and value is index
target_token_to_index_dict = {}

for k, v in enumerate(Target_Tokens):
    target_index_to_token_dict[k] = v
    target_token_to_index_dict[v] = k
    
    
max_len_source_seq = max([len(line) for line in Source_Sequences])
max_len_target_seq = max([len(line) for line in Target_Sequences])

print('max length source: ',max_len_source_seq)
print('source vocab: ',len(Source_Tokens))
print('max length target: ',max_len_target_seq)
print('target vocab: ',len(Target_Tokens))
